# Generate the CSV file according to Demo

In [2]:
import pandas as pd
import json
from datetime import datetime

# Function to convert period to date range
def period_to_date(year, period):
    if period == 'Q1':
        return f"{year}-01-01", f"{year}-03-31"
    elif period == 'Q2':
        return f"{year}-04-01", f"{year}-06-30"
    elif period == 'Q3':
        return f"{year}-07-01", f"{year}-09-30"
    elif period == 'Q4':
        return f"{year}-10-01", f"{year}-12-31"
    return f"{year}-01-01", f"{year}-12-31"

# Function to parse date and format period like "January 12, 24"
def format_label(date_str):
    try:
        date = pd.to_datetime(date_str)
        return f"{date.strftime('%B %d, %y')}"
    except (ValueError, TypeError):
        return "Unknown Date"

# Function to process JSON into CSV
def process_json_to_csv(json_file, output_csv, default_symbol="1101.TW"):
    try:
        with open(json_file, 'r') as f:
            data = json.load(f)
        
        print(f"Loaded {json_file}: Keys = {list(data.keys())}")

        # Debug historicalPriceFull
        if 'historicalPriceFull' in data:
            if isinstance(data['historicalPriceFull'], dict):
                print(f"historicalPriceFull is a dict with keys: {list(data['historicalPriceFull'].keys())}")
                if 'symbol' in data['historicalPriceFull'] and 'historical' in data['historicalPriceFull']:
                    symbol = data['historicalPriceFull']['symbol']
                    historical_data = data['historicalPriceFull']['historical']
                    if isinstance(historical_data, list):
                        print(f"historicalPriceFull['historical'] is a list with {len(historical_data)} records.")
                        if historical_data:
                            temp_df = pd.DataFrame(historical_data)
                            print(f"Columns in historicalPriceFull['historical']: {list(temp_df.columns)}")
                        else:
                            print("historicalPriceFull['historical'] is an empty list.")
                    else:
                        print(f"historicalPriceFull['historical'] is not a list: {type(historical_data)}")
                else:
                    print("historicalPriceFull does not contain 'symbol' and 'historical' keys.")
            else:
                print(f"historicalPriceFull is not a dict: {type(data['historicalPriceFull'])}")
        else:
            print("historicalPriceFull key not found in JSON.")

        # Debug tech5
        if 'tech5' in data:
            if isinstance(data['tech5'], list):
                print(f"tech5 is a list with {len(data['tech5'])} records.")
                if data['tech5']:
                    temp_df = pd.DataFrame(data['tech5'])
                    print(f"Columns in tech5: {list(temp_df.columns)}")
                else:
                    print("tech5 is an empty list.")
            else:
                print(f"tech5 is not a list: {type(data['tech5'])}")
        else:
            print("tech5 key not found in JSON.")

        # Process financial data to map to daily dates
        financial_categories = ['financialGrowth', 'ratios', 'cashFlowStatementGrowth', 
                                'incomeStatementGrowth', 'balanceSheetStatementGrowth']
        financial_dfs = {}
        for category in financial_categories:
            if category in data and isinstance(data[category], list):
                df = pd.DataFrame(data[category])
                if not df.empty:
                    if 'symbol' not in df.columns:
                        print(f"Warning: 'symbol' column missing in '{category}'. Skipping this dataset.")
                        continue
                    if all(col in df.columns for col in ['calendarYear', 'period']):
                        df[['start_date', 'end_date']] = df.apply(
                            lambda row: period_to_date(row['calendarYear'], row['period']),
                            axis=1, result_type='expand'
                        )
                        df['start_date'] = pd.to_datetime(df['start_date'])
                        df['end_date'] = pd.to_datetime(df['end_date'])
                        financial_dfs[category] = df
                    else:
                        print(f"Warning: '{category}' missing 'calendarYear' or 'period'. Skipping date processing.")
                        financial_dfs[category] = df
                else:
                    print(f"Warning: '{category}' is empty.")

        # Merge financial data
        merged_financial = pd.DataFrame()
        for category, df in financial_dfs.items():
            try:
                df_grouped = df.groupby(['symbol', 'start_date', 'end_date']).first().reset_index()
                if merged_financial.empty:
                    merged_financial = df_grouped
                else:
                    cols_to_drop = [col for col in df_grouped.columns if col in merged_financial.columns and col not in ['symbol', 'start_date', 'end_date']]
                    merged_financial = merged_financial.merge(df_grouped.drop(columns=cols_to_drop), on=['symbol', 'start_date', 'end_date'], how='outer')
            except Exception as e:
                print(f"Error merging '{category}': {str(e)}")

        # Drop 'date', 'calendarYear', and 'period' from merged_financial to avoid conflicts
        if not merged_financial.empty:
            cols_to_drop = ['date', 'calendarYear', 'period']
            merged_financial = merged_financial.drop(columns=[col for col in cols_to_drop if col in merged_financial.columns])

        # Process daily data (prefer historicalPriceFull['historical'])
        daily_df = pd.DataFrame()
        daily_source = None
        if 'historicalPriceFull' in data and isinstance(data['historicalPriceFull'], dict):
            if 'symbol' in data['historicalPriceFull'] and 'historical' in data['historicalPriceFull']:
                symbol = data['historicalPriceFull']['symbol']
                historical_data = data['historicalPriceFull']['historical']
                if isinstance(historical_data, list) and historical_data:
                    daily_df = pd.DataFrame(historical_data)
                    daily_df['symbol'] = symbol  # Add symbol column
                    daily_source = "historicalPriceFull['historical']"

        if daily_df.empty and 'tech5' in data and isinstance(data['tech5'], list) and data['tech5']:
            daily_df = pd.DataFrame(data['tech5'])
            daily_df = daily_df.rename(columns=lambda x: x.replace('_tech5', ''))
            daily_source = 'tech5'

        if daily_df.empty:
            print("No valid daily data found in 'historicalPriceFull' or 'tech5'. CSV not created.")
            return pd.DataFrame()

        # Debug: Print columns of daily data
        print(f"Columns in {daily_source} before processing: {list(daily_df.columns)}")

        # Check for 'date' column
        if 'date' not in daily_df.columns:
            print(f"Error: 'date' column missing in {daily_source}. Cannot proceed.")
            return pd.DataFrame()

        # Check for 'symbol' in daily data
        if 'symbol' not in daily_df.columns:
            print(f"Warning: 'symbol' column missing in {daily_source}. Adding default symbol '{default_symbol}'.")
            daily_df['symbol'] = default_symbol

        # Compute derived columns
        daily_df = daily_df.dropna(subset=['date'])
        daily_df['date'] = pd.to_datetime(daily_df['date'], errors='coerce')
        daily_df = daily_df.dropna(subset=['date'])
        print(f"Columns after date conversion: {list(daily_df.columns)}")
        
        # Only compute calendarYear and period, as others are already in historicalPriceFull
        daily_df['calendarYear'] = daily_df['date'].dt.year
        daily_df['period'] = daily_df['date'].apply(format_label)

        # Debug: Print columns after derived columns
        print(f"Columns after derived columns: {list(daily_df.columns)}")

        # Merge financial data with daily data
        final_df = daily_df.copy()
        if not merged_financial.empty:
            print(f"Merging financial data. Columns in merged_financial: {list(merged_financial.columns)}")
            merged_financial = merged_financial.sort_values('start_date')
            final_df = final_df.merge(merged_financial, on='symbol', how='left')
            print(f"Columns after merge: {list(final_df.columns)}")
            if 'start_date' in final_df.columns and 'end_date' in final_df.columns:
                final_df = final_df[
                    (final_df['date'] >= final_df['start_date']) & 
                    (final_df['date'] <= final_df['end_date'])
                ]
                final_df = final_df.drop(columns=['start_date', 'end_date'], errors='ignore')
            print(f"Columns after filtering: {list(final_df.columns)}")

        # Select columns to match demo CSV
        demo_columns = ['date', 'symbol', 'open', 'high', 'low', 'close', 'adjClose', 
                        'volume', 'unadjustedVolume', 'change', 'changePercent', 'vwap', 
                        'label', 'changeOverTime', 'calendarYear', 'period']
        final_df = final_df[demo_columns + [col for col in final_df.columns if col not in demo_columns]]

        # Save to CSV
        final_df.to_csv(output_csv, index=False)
        print(f"CSV file saved as {output_csv}")
        return final_df

    except Exception as e:
        print(f"Error: {str(e)}")
        return pd.DataFrame()

# Usage
json_file = 'output_clean_date_technical.json'
output_csv = 'financial_data_by_symbol.csv'
result_df = process_json_to_csv(json_file, output_csv)

# Read and display the CSV
if not result_df.empty:
    print("\nFirst 5 rows of the CSV:")
    print(result_df.head())

Loaded output_clean_date_technical.json: Keys = ['financialGrowth', 'ratios', 'cashFlowStatementGrowth', 'incomeStatementGrowth', 'balanceSheetStatementGrowth', 'historicalPriceFull', 'tech5', 'tech20', 'tech60', 'tech252']
historicalPriceFull is a dict with keys: ['symbol', 'historical']
historicalPriceFull['historical'] is a list with 736 records.
Columns in historicalPriceFull['historical']: ['date', 'open', 'high', 'low', 'close', 'adjClose', 'volume', 'unadjustedVolume', 'change', 'changePercent', 'vwap', 'label', 'changeOverTime']
tech5 is a list with 736 records.
Columns in tech5: ['date', 'open', 'high', 'low', 'close', 'volume', 'sma', 'ema', 'wma', 'dema', 'tema', 'williams', 'rsi', 'adx', 'standardDeviation']
Columns in historicalPriceFull['historical'] before processing: ['date', 'open', 'high', 'low', 'close', 'adjClose', 'volume', 'unadjustedVolume', 'change', 'changePercent', 'vwap', 'label', 'changeOverTime', 'symbol']
Columns after date conversion: ['date', 'open', 'hi

In [3]:
import pandas as pd

csv_file = 'financial_data_by_symbol.csv'

df = pd.read_csv(csv_file)
df

,date,symbol,open,high,low,close,adjClose,volume,unadjustedVolume,change,...,growthTotalLiabilities,growthCommonStock,growthRetainedEarnings,growthAccumulatedOtherComprehensiveIncomeLoss,growthOthertotalStockholdersEquity,growthTotalStockholdersEquity,growthTotalLiabilitiesAndStockholdersEquity,growthTotalInvestments,growthTotalDebt,growthNetDebt
0,2023-09-28,1101.TW,33.10,33.30,33.05,33.25,33.250000,18217658,18217658,0.15,...,0.029640,0.000000,0.018448,0,0.093871,0.037866,0.036323,0.042211,0.053001,-0.110068
1,2023-09-27,1101.TW,33.00,33.20,32.90,33.05,33.050000,15043077,15043077,0.05,...,0.029640,0.000000,0.018448,0,0.093871,0.037866,0.036323,0.042211,0.053001,-0.110068
2,2023-09-26,1101.TW,33.10,33.30,33.00,33.00,33.000000,18175159,18175159,-0.10,...,0.029640,0.000000,0.018448,0,0.093871,0.037866,0.036323,0.042211,0.053001,-0.110068
3,2023-09-25,1101.TW,33.40,33.45,33.00,33.10,33.100000,29278663,29278663,-0.30,...,0.029640,0.000000,0.018448,0,0.093871,0.037866,0.036323,0.042211,0.053001,-0.110068
4,2023-09-22,1101.TW,33.50,33.60,33.35,33.50,33.500000,23115549,23115549,0.00,...,0.029640,0.000000,0.018448,0,0.093871,0.037866,0.036323,0.042211,0.053001,-0.110068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
659,2021-01-20,1101.TW,37.87,37.91,37.09,37.23,33.397839,38529978,38529978,-0.64,...,-0.036521,0.025332,0.039989,0,0.031521,0.032529,0.001455,-0.004937,-0.081935,-0.248028
660,2021-01-19,1101.TW,37.82,38.18,37.82,37.96,34.050301,13261660,13261660,0.14,...,-0.036521,0.025332,0.039989,0,0.031521,0.032529,0.001455,-0.004937,-0.081935,-0.248028
661,2021-01-18,1101.TW,38.14,38.14,37.64,37.82,33.927963,27598255,27598255,-0.32,...,-0.036521,0.025332,0.039989,0,0.031521,0.032529,0.001455,-0.004937,-0.081935,-0.248028
662,2021-01-15,1101.TW,38.64,38.64,38.14,38.14,34.213417,33681520,33681520,-0.50,...,-0.036521,0.025332,0.039989,0,0.031521,0.032529,0.001455,-0.004937,-0.081935,-0.248028


In [4]:
df['symbol'].unique()

array(['1101.TW'], dtype=object)

In [1]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
import numpy as np

df = pd.read_csv('financial_data_by_symbol.csv')
print("CSV loaded successfully.")


# Clean column names (fix 'growth\nNetDebt')
df.columns = df.columns.str.replace('\n', '')

# Print columns to debug
print(f"Columns in the dataset: {list(df.columns)}")

# Check for 'close' column (case-insensitive)
close_col = None
for col in df.columns:
    if col.lower() == 'close':
        close_col = col
        break

# If 'close' is not found, try 'adjClose'
if close_col is None:
    for col in df.columns:
        if col.lower() == 'adjclose':
            close_col = col
            print(f"'close' column not found. Using '{close_col}' as a fallback.")
            break

# If neither 'close' nor 'adjClose' is found, try 'close_tech5', 'close_tech20', etc.
if close_col is None:
    for col in df.columns:
        if col.lower() in ['close_tech5', 'close_tech20', 'close_tech60', 'close_tech252']:
            close_col = col
            print(f"'close' and 'adjClose' not found. Using '{close_col}' as a fallback.")
            break

# If no suitable column is found, raise an error
if close_col is None:
    print("Error: No suitable closing price column found ('close', 'adjClose', 'close_tech5', etc.). Available columns:")
    print(list(df.columns))
    exit(1)

print(f"Using column '{close_col}' for closing price.")

# Create target variable: 1 if close price increases by 10% in 90 days, 0 otherwise
df['future_close'] = df[close_col].shift(-90)
df['target'] = (df['future_close'] >= df[close_col] * 1.10).astype(int)
df = df.dropna(subset=['target'])
print(f"Target variable created. Number of rows after dropping NaNs in target: {len(df)}")

# Handle missing values
df = df.fillna(method='ffill')  # Forward-fill financial metrics
df = df.fillna(df.median(numeric_only=True))  # Impute remaining NaNs with median
print("Missing values handled.")

# Feature engineering (create all features at once to avoid fragmentation)
new_features = {
    'close_lag1': df[close_col].shift(1),
    'close_lag5': df[close_col].shift(5),
    'close_ma5': df[close_col].rolling(window=5).mean(),
    'close_ma20': df[close_col].rolling(window=20).mean(),
    'volatility_20': df[close_col].rolling(window=20).std(),
    'momentum_10': df[close_col].pct_change(periods=10)
}

# Compute RSI
def compute_rsi(data, periods=14):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=periods).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=periods).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))
new_features['rsi'] = compute_rsi(df[close_col])

# Add MACD
new_features['ema12'] = df[close_col].ewm(span=12, adjust=False).mean()
new_features['ema26'] = df[close_col].ewm(span=26, adjust=False).mean()
new_features['macd'] = new_features['ema12'] - new_features['ema26']
new_features['signal_line'] = new_features['macd'].ewm(span=9, adjust=False).mean()

# Add Bollinger Bands
new_features['bb_middle'] = df[close_col].rolling(window=20).mean()
new_features['bb_std'] = df[close_col].rolling(window=20).std()
new_features['bb_upper'] = new_features['bb_middle'] + 2 * new_features['bb_std']
new_features['bb_lower'] = new_features['bb_middle'] - 2 * new_features['bb_std']

# Add new features to DataFrame at once
df = pd.concat([df, pd.DataFrame(new_features)], axis=1)

# Drop rows with NaN values after feature engineering
df = df.dropna()
print(f"Number of rows after feature engineering and dropping NaNs: {len(df)}")

# Select features
features = [col for col in df.columns if col not in ['date', 'symbol', 'label', 'period', 'future_close', 'target', 'start_date', 'end_date']]
X = df[features]
y = df['target']
print(f"Features selected: {features}")

# Stratified train-test split to ensure both classes are represented
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print(f"Training set size: {len(X_train)}, Test set size: {len(X_test)}")
print(f"Training set class distribution:\n{y_train.value_counts()}")
print(f"Test set class distribution:\n{y_test.value_counts()}")

# Handle class imbalance with SMOTE
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)
print(f"After SMOTE, training set size: {len(X_train)}")
print(f"Training set class distribution after SMOTE:\n{y_train.value_counts()}")

# Convert to DMatrix for xgboost.train with feature names
dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=features)
dtest = xgb.DMatrix(X_test, label=y_test, feature_names=features)

# Define parameters with scale_pos_weight to handle class imbalance
scale_pos_weight = sum(y_train == 0) / sum(y_train == 1) if sum(y_train == 1) > 0 else 1
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'max_depth': 5,
    'eta': 0.1,
    'scale_pos_weight': scale_pos_weight
}

# Train with callbacks using xgboost.train
try:
    model = xgb.train(
        params=params,
        dtrain=dtrain,
        num_boost_round=100,
        evals=[(dtest, 'test')],
        callbacks=[
            xgb.callback.EarlyStopping(
                rounds=10,
                metric_name='logloss',
                save_best=True,
                maximize=False
            )
        ],
        verbose_eval=False
    )
    print("Model training completed.")
except Exception as e:
    print(f"Error during model training: {str(e)}")
    exit(1)

# Predict with a lower threshold to improve recall for class 1
y_pred_prob = model.predict(dtest)
y_pred = (y_pred_prob > 0.3).astype(int)  # Lower threshold from 0.5 to 0.3

# Evaluate
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Feature importance
feature_importance = model.get_score(importance_type='gain')
importance_df = pd.DataFrame({
    'Feature': list(feature_importance.keys()),
    'Importance': list(feature_importance.values())
})
importance_df = importance_df.sort_values(by='Importance', ascending=False)
print("\nTop 10 Key Impact Features:")
print(importance_df.head(10))

# Hyperparameter tuning (we'll use XGBClassifier for GridSearchCV since it integrates better with scikit-learn)
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}
grid_search = GridSearchCV(
    estimator=xgb.XGBClassifier(
        objective='binary:logistic',
        eval_metric='logloss',
        scale_pos_weight=scale_pos_weight
    ),
    param_grid=param_grid,
    cv=3,
    scoring='f1',
    verbose=1
)
grid_search.fit(X_train, y_train)
print("Best parameters:", grid_search.best_params_)
model_sklearn = grid_search.best_estimator_

# Re-evaluate with the tuned model
y_pred = model_sklearn.predict(X_test)
print("\nClassification Report (after tuning):")
print(classification_report(y_test, y_pred))

CSV loaded successfully.
Columns in the dataset: ['date', 'symbol', 'open', 'high', 'low', 'close', 'adjClose', 'volume', 'unadjustedVolume', 'change', 'changePercent', 'vwap', 'label', 'changeOverTime', 'calendarYear', 'period', 'revenueGrowth', 'grossProfitGrowth', 'ebitgrowth', 'operatingIncomeGrowth', 'netIncomeGrowth', 'epsgrowth', 'epsdilutedGrowth', 'weightedAverageSharesGrowth', 'weightedAverageSharesDilutedGrowth', 'dividendsperShareGrowth', 'operatingCashFlowGrowth', 'freeCashFlowGrowth', 'tenYRevenueGrowthPerShare', 'fiveYRevenueGrowthPerShare', 'threeYRevenueGrowthPerShare', 'tenYOperatingCFGrowthPerShare', 'fiveYOperatingCFGrowthPerShare', 'threeYOperatingCFGrowthPerShare', 'tenYNetIncomeGrowthPerShare', 'fiveYNetIncomeGrowthPerShare', 'threeYNetIncomeGrowthPerShare', 'tenYShareholdersEquityGrowthPerShare', 'fiveYShareholdersEquityGrowthPerShare', 'threeYShareholdersEquityGrowthPerShare', 'tenYDividendperShareGrowthPerShare', 'fiveYDividendperShareGrowthPerShare', 'threeYD